In [ ]:
#Updated Strategy
#To address the unchanged perplexity, we’ll update the code to:

#Remove Stopword Filtering: Increase vocabulary size and context.
#Test Stemming Instead of Lemmatization: Reduce vocabulary size more aggressively.
#Use Simple split() Tokenization: Match the original code’s approach.
#Increase num_merges to 1000: Improve subword coverage.
#Fix Validation Tokenization: Ensure subword units are consistent across training and validation.
#Add Backoff Smoothing for Bigrams: Improve handling of unseen bigrams.
#Enhance Debugging: Print top subword units and their frequencies to verify BPE effectiveness.

In [7]:
# Assignment1_complete.py

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import defaultdict, Counter
import math

# Download NLTK resources (run once)
nltk.download('punkt')
stemmer = PorterStemmer()

def preprocess_text(text):
    """
    Preprocess the input text by:
    1. Lowercasing
    2. Tokenization with split()
    3. Keeping alphanumeric tokens
    4. Stemming with PorterStemmer
    """
    text = text.lower()
    tokens = text.strip().split()
    tokens = [word for word in tokens if word.isalnum()]
    tokens = [stemmer.stem(word) for word in tokens]
    return tokens

# Preprocessing for training set
training = 'train.txt'
tokens = []

with open(training, 'r', encoding='utf-8') as t:
    for line in t:
        processed_tokens = preprocess_text(line.strip())
        tokens_per_line = ['<s>'] + processed_tokens + ['</s>']
        tokens.append(tokens_per_line)

# Preprocessing for validation set
validation = 'val.txt'
validation_tokens = []

with open(validation, 'r', encoding='utf-8') as v:
    for line in v:
        processed_tokens = preprocess_text(line.strip())
        tokens_per_line = ['<s>'] + processed_tokens + ['</s>']
        validation_tokens.append(tokens_per_line)

# Unigram model
unigram_counts = {}
total_word_count = 0
for tokens_per_line in tokens:
    for token in tokens_per_line:
        if token not in unigram_counts:
            unigram_counts[token] = 1
        else:
            unigram_counts[token] += 1
        total_word_count += 1

unigram_model = {}
for word, count in unigram_counts.items():
    unigram_model[word] = count / total_word_count

unsmoothened_unigram_model = unigram_model.copy()

# Debugging: Print vocabulary size
print(f"Training vocabulary size: {len(unigram_counts)}")

# Debugging: Validation <unk> frequency
unk_count_val = sum(1 for line in validation_tokens for token in line if token not in unigram_counts and token not in ['<s>', '</s>'])
print(f"Frequency of <unk> in validation set (before subword): {unk_count_val}")

# Bigram model with fix for KeyError
bigram_counts = {}
for tokens_per_line in tokens:
    for i in range(len(tokens_per_line) - 1):
        bigram = (tokens_per_line[i], tokens_per_line[i + 1])
        if bigram not in bigram_counts:
            bigram_counts[bigram] = 1
        else:
            bigram_counts[bigram] += 1

bigram_model = {}
for bigram, count in bigram_counts.items():
    first_word = bigram[0]
    if first_word in unigram_counts:
        bigram_model[bigram] = count / unigram_counts[first_word]
    else:
        print(f"Warning: Skipping bigram {bigram} because '{first_word}' not found in unigram_counts")

unsmoothened_bigram_model = bigram_model.copy()

# First method to handle unknown words
common_words = {}
for word, count in unigram_counts.items():
    if count > 1:
        common_words[word] = count

tokens_with_unk = tokens.copy()
for tokens_per_line in tokens_with_unk:
    for i in range(len(tokens_per_line)):
        if tokens_per_line[i] not in common_words:
            tokens_per_line[i] = '<unk>'

# Unigram model with <unk>
unigram_counts_with_unk = {}
total_word_count_with_unk = 0
for tokens_per_line in tokens_with_unk:
    for token in tokens_per_line:
        if token not in unigram_counts_with_unk:
            unigram_counts_with_unk[token] = 1
        else:
            unigram_counts_with_unk[token] += 1
        total_word_count_with_unk += 1

unigram_model = {}
for unigram, count in unigram_counts_with_unk.items():
    unigram_model[unigram] = count / total_word_count_with_unk

unigram_model_with_unk = unigram_model.copy()

# Debugging: Print <unk> frequency
print(f"Frequency of <unk> in training set: {unigram_counts_with_unk.get('<unk>', 0)}")

# Bigram model with <unk>
bigram_counts_with_unk = {}
for tokens_per_line in tokens_with_unk:
    for i in range(len(tokens_per_line) - 1):
        bigram = (tokens_per_line[i], tokens_per_line[i + 1])
        if bigram not in bigram_counts_with_unk:
            bigram_counts_with_unk[bigram] = 1
        else:
            bigram_counts_with_unk[bigram] += 1

bigram_model = {}
for bigram, count in bigram_counts_with_unk.items():
    first_word = bigram[0]
    bigram_model[bigram] = count / unigram_counts_with_unk[first_word]

bigram_model_with_unk = bigram_model.copy()

# Second unknown words handling: Subword tokenization (BPE)
def get_stats(vocab):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = defaultdict(int)
for line in tokens:
    for token in line[1:-1]:
        formatted = ' '.join(list(token)) + ' </w>'
        vocab[formatted] += 1

num_merges = 1000  # Increased from 500
merges = {}
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    merges[best] = i
    vocab = merge_vocab(best, vocab)

def tokenize(word, merges):
    symbols = list(word)
    while True:
        pair = None
        min_rank = float('inf')
        idx = -1
        for i in range(len(symbols) - 1):
            p = (symbols[i], symbols[i+1])
            if p in merges:
                rank = merges[p]
                if rank < min_rank:
                    min_rank = rank
                    pair = p
                    idx = i
        if pair is None:
            break
        new_symbol = pair[0] + pair[1]
        symbols = symbols[:idx] + [new_symbol] + symbols[idx+2:]
    return symbols

# Debugging: Print sample subword tokenizations and top subwords
sample_words = ['priceline', 'travelzoo', 'booked']
print("Sample subword tokenizations:")
for word in sample_words:
    print(f"{word} -> {tokenize(word, merges)}")

# Apply subword tokenization to training set
tokens_with_subword = [line.copy() for line in tokens]
for line in tokens_with_subword:
    i = 0
    while i < len(line):
        token = line[i]
        if token not in common_words and token not in ['<s>', '</s>']:
            subwords = tokenize(token, merges)
            line[i:i+1] = subwords
            i += len(subwords)
        else:
            i += 1

# Unigram model with subword
unigram_counts_with_subword = {}
total_word_count_with_subword = 0
for tokens_per_line in tokens_with_subword:
    for token in tokens_per_line:
        if token not in unigram_counts_with_subword:
            unigram_counts_with_subword[token] = 1
        else:
            unigram_counts_with_subword[token] += 1
        total_word_count_with_subword += 1

# Add <unk> to subword vocabulary
unigram_counts_with_subword['<unk>'] = 1
total_word_count_with_subword += 1

unigram_model_subword = {}
for word, count in unigram_counts_with_subword.items():
    unigram_model_subword[word] = count / total_word_count_with_subword

# Debugging: Print top 5 subword units
print("Top 5 subword units:")
for word, count in sorted(unigram_counts_with_subword.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{word}: {count}")

# Bigram model with subword
bigram_counts_with_subword = {}
for tokens_per_line in tokens_with_subword:
    for i in range(len(tokens_per_line)-1):
        bigram = (tokens_per_line[i], tokens_per_line[i+1])
        if bigram not in bigram_counts_with_subword:
            bigram_counts_with_subword[bigram] = 1
        else:
            bigram_counts_with_subword[bigram] += 1

bigram_model_with_subword = {}
for bigram, count in bigram_counts_with_subword.items():
    first_word = bigram[0]
    if first_word in unigram_counts_with_subword:
        bigram_model_with_subword[bigram] = count / unigram_counts_with_subword[first_word]
    else:
        print(f"Warning: Skipping bigram {bigram} because '{first_word}' not found in unigram_counts_with_subword")

# Apply subword tokenization to validation set
validation_tokens_with_subword = [line.copy() for line in validation_tokens]
for line in validation_tokens_with_subword:
    i = 0
    while i < len(line):
        token = line[i]
        if token not in common_words and token not in ['<s>', '</s>']:
            subwords = tokenize(token, merges)
            line[i:i+1] = subwords
            i += len(subwords)
        else:
            i += 1

# Debugging: Print subword vocabulary size and validation <unk> frequency
print(f"Subword training vocabulary size: {len(unigram_counts_with_subword)}")
unk_count_val_subword = sum(1 for line in validation_tokens_with_subword for token in line if token not in unigram_counts_with_subword and token not in ['<s>', '</s>'])
print(f"Frequency of <unk> in validation set (after subword): {unk_count_val_subword}")

# Laplace smoothing
unigram_model = {}
for word, count in unigram_counts_with_unk.items():
    unigram_model[word] = (count + 1) / (total_word_count_with_unk + len(unigram_counts_with_unk))
unigram_model_laplace = unigram_model.copy()

bigram_model = {}
for bigram, count in bigram_counts_with_unk.items():
    first_word = bigram[0]
    bigram_model[bigram] = (count + 1) / (unigram_counts_with_unk[first_word] + len(unigram_counts_with_unk))
bigram_model_laplace = bigram_model.copy()

# Laplace smoothing for subword
unigram_model_subword = {}
for word, count in unigram_counts_with_subword.items():
    unigram_model_subword[word] = (count + 1) / (total_word_count_with_subword + len(unigram_counts_with_subword))
unigram_model_laplace_subword = unigram_model_subword.copy()

bigram_model_subword = {}
for bigram, count in bigram_counts_with_subword.items():
    first_word = bigram[0]
    bigram_model_subword[bigram] = (count + 1) / (unigram_counts_with_subword[first_word] + len(unigram_counts_with_subword))
bigram_model_laplace_subword = bigram_model_subword.copy()

# Backoff smoothing for bigram
def bigram_backoff_perplexity_calculator(validation_data, bigram_model, unigram_model, vocabulary):
    V = len(set(vocabulary.keys()))
    log_prob_sum = 0.0
    N = 0
    alpha = 0.4  # Backoff weight
    for review in validation_data:
        review = [word if word in vocabulary else '<unk>' for word in review]
        for i in range(len(review) - 1):
            bigram = (review[i], review[i + 1])
            if bigram in bigram_model:
                probability = bigram_model[bigram]
            else:
                probability = alpha * unigram_model.get(review[i + 1], 1 / (total_word_count + V))
            N += 1
            log_prob_sum += math.log(probability)
        N += len(review) - 1
    if N == 0:
        return float('inf')
    perplexity = math.exp(-log_prob_sum / N)
    return perplexity

# Perplexity calculation for unigram model
def perplexity_calculator(validation_data, model, vocabulary):
    log_prob_sum = 0.0
    N = 0
    for review_tokens in validation_data:
        for token in review_tokens:
            if token == '<s>':
                continue
            N += 1
            lookup_token = token
            if lookup_token not in vocabulary:
                lookup_token = '<unk>'
            probability = model.get(lookup_token, 1 / (total_word_count + len(vocabulary)))
            log_prob_sum += math.log(probability)
    avg_log_prob = log_prob_sum / N
    perplexity = math.exp(-avg_log_prob)
    return perplexity

# Print perplexity results
print(f"Perplexity for unigram model (Laplace) on {validation}: {perplexity_calculator(validation_tokens, unigram_model_laplace, unigram_counts_with_unk)}")
print(f"Perplexity for bigram model (Backoff) on {validation}: {bigram_backoff_perplexity_calculator(validation_tokens, bigram_model_laplace, unigram_model_laplace, unigram_counts_with_unk)}")
print(f"Perplexity for unigram model (Laplace) with subword on {validation}: {perplexity_calculator(validation_tokens_with_subword, unigram_model_laplace_subword, unigram_counts_with_subword)}")
print(f"Perplexity for bigram model (Backoff) with subword on {validation}: {bigram_backoff_perplexity_calculator(validation_tokens_with_subword, bigram_model_laplace_subword, unigram_model_laplace_subword, unigram_counts_with_subword)}")

[nltk_data] Downloading package punkt to /Users/shamitha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Training vocabulary size: 4301
Frequency of <unk> in validation set (before subword): 162
Frequency of <unk> in training set: 1830
Sample subword tokenizations:
priceline -> ['pricel', 'in', 'e']
travelzoo -> ['trav', 'el', 'z', 'oo']
booked -> ['b', 'oo', 'k', 'e', 'd']
Top 5 subword units:
the: 5292
and: 2593
a: 2246
to: 2090
wa: 1826
Subword training vocabulary size: 2472
Frequency of <unk> in validation set (after subword): 601
Perplexity for unigram model (Laplace) on val.txt: 325.44265502170816
Perplexity for bigram model (Backoff) on val.txt: 21.358844475560957
Perplexity for unigram model (Laplace) with subword on val.txt: 515.9757944669111
Perplexity for bigram model (Backoff) with subword on val.txt: 22.13630860092163
